In [1]:
from calendar_clean_exploration import clean_calendar
import datetime,calendar
import pandas as pd

def holiday(year):
    #fixed holidays
    fixmd = [[1,1],[7,4],[11,11]]
    holidays = [datetime.datetime(year,x[0],x[1]) for x in fixmd]
    #floating holidays
    #'2,3,1': the third Monday of Feburary;'5,-1,1': the last Monday of May 
    floatmwd = ['1,3,1','2,3,1','5,-1,1','9,1,1','10,2,1','11,4,4']
    holidays.extend([flholiday(year,x) for x in floatmwd])
    return sorted(holidays)
    
def flholiday(y,mwd):
    #input year and month-week-weekday
    m,w,d = list(map(int,mwd.split(',')))
    d = d-1
    wday, mrange = calendar.monthrange(y, m) #firstDayWeekDay,monthRange
    if w>0:
        if wday<=d:
            day = 1+7*(w-1)+d-wday
        else:
            day = 1+7*w-(wday-d)
    else:
        wday = datetime.datetime(y,m,mrange).weekday()
        if d<=wday:
            day = mrange+7*(w+1)-(wday-d)
        else:
            day = mrange+7*w+(d-wday)
    return datetime.datetime(y,m,day)   

df = clean_calendar('../data/2018_04_14/calendar.csv')
#return 0-6, stands for Mon to Sun
df['week'] = df.date.dt.dayofweek

#day of week
week = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
for i,w in enumerate(week):
    df[w]=0
    df.loc[df['week']==i,w]=1
    
#which month
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i,m in enumerate(month):
    df[m]=0
    df.loc[df.date.dt.month==i+1,m]=1

#which year
year = [2018,2019]
for y in year:
    df[y]=0
    df.loc[df.date.dt.year==y,y]=1

#christams holidays: 12/25-12/31
df['ChristmasHolidays']=0
df.loc[(df.date.dt.month==12) & (df.date.dt.day>=25),'ChristmasHolidays']=1

#other holidays
holidays = ['NewYear','MartinLK','President','Memorial','Independence','Labor','Columbus','Veterans','Thanksgiving']
holiday18 = holiday(2018)
holiday19 = holiday(2019)
for i,h in enumerate(holidays):
    df[h]=0
    df.loc[df.date==holiday18[i],h]=1
    df.loc[df.date==holiday19[i],h]=1
    
df.drop(columns=['week','date'],inplace=True)

In [62]:
holidays.append('ChristmasHolidays')
df_nholi = df[(df[holidays]==0).all(axis=1)].drop(columns=holidays)
listing = df_nholi.listing_id.unique()

In [67]:
gmark = ['listing_id']
gmark.extend(year)
gmark.extend(month)
gmark.extend(week)

result = df_nholi.groupby(gmark)['price'].mean().to_frame()
result.reset_index(inplace=True)

In [70]:
df_holi = df[(df[holidays]==1).any(axis=1)]

In [75]:
df = pd.concat([df_holi,result],ignore_index=True,sort=False).fillna(0)